<div style="border:solid green 2px; padding: 20px">
<b>Привет, Александр!</b>

Меня зовут Александр Пономаренко, и я буду проверять твой проект. Предлагаю общаться на «ты» :) Но если это не удобно - дай знать, и мы перейдем на "вы". 

Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе стать аналитиком данных. Ты уже проделал большую работу над проектом, но давай сделаем его еще лучше. Ниже ты найдешь мои комментарии - **пожалуйста, не перемещай, не изменяй и не удаляй их**. Увидев у тебя ошибку, в первый раз я лишь укажу на ее наличие и дам тебе возможность самой найти и исправить ее. На реальной работе твой начальник будет поступать так же, а я пытаюсь подготовить тебя именно к работе аналитиком. Но если ты пока не справишься с такой задачей - при следующей проверке я дам более точную подсказку. Я буду использовать цветовую разметку:

<div class="alert alert-danger">
<b>Комментарий ревьюера ❌:</b> Так выделены самые важные замечания. Без их отработки проект не будет принят. </div>

<div class="alert alert-warning">
<b>Комментарий ревьюера ⚠️:</b> Так выделены небольшие замечания. Я надеюсь, что их ты тоже учтешь - твой проект от этого станет только лучше. Но настаивать на их отработке не буду.

</div>

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Так я выделяю все остальные комментарии.</div>

Давай работать над проектом в диалоге: **если ты что-то меняешь в проекте или отвечаешь на мои комменатри — пиши об этом.** Мне будет легче отследить изменения, если ты выделишь свои комментарии:
<div class="alert alert-info"> <b>Комментарий студента:</b> Например, вот так.</div>

Всё это поможет выполнить повторную проверку твоего проекта оперативнее. 

<div class="alert alert-info"> <b>Комментарий студента:</b> Привет, Александр! На "ты" норм) </div>

# Предсказание стоимости жилья

Данные, которые мы будем анализировать, были собраны в рамках переписи населения в США. Каждая строка содержит агрегированную статистику о жилом массиве. Жилой массив — минимальная географическая единица с населением от 600 до 3000 человек в зависимости от штата. Одна строка в данных содержит статистику в среднем о 1425.5 обитателях жилого массива.

В проекте нам нужно создать модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных предскажем медианную стоимость дома в жилом массиве. Обучим модель и сделаем предсказания на тестовой выборке. Для оценки качества модели будем использовать метрики RMSE, MAE и R2.

## Подготовка данных

### Загрузка данных

Импортируем нужные библиотеки:

In [1]:
import pandas as pd 
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression

from pyspark.ml.feature import OneHotEncoder 

from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
        
RANDOM_SEED = 12

Инициализируем локальную Spark-сессию:

In [2]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("California_Housing") \
                    .getOrCreate()

Загружаем файл с данными:

In [3]:
df = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema = True) 
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



**Вывод:** данные успешно загружены. Типы данных - float, корме одного столбца(ocean_proximity) - str. Все стобцы хранят количественные данные, опять же кроме одного *ocean_proximity* - категориальная переменная.

Посморим на несколько строк полученного дата-фрейма:

In [4]:
df.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

**Вывод:** таблица загружена, названия колонок читабельны. Но две колонки накладываются друг на друга.


Представим изображение таблицы в формате pandas, для более удобного восприятия:

In [5]:
df_pd = df.toPandas()
df_pd.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


-В колонках датасета содержатся следующие данные:
+ longitude — широта;
+ latitude — долгота;
+ housing_median_age — медианный возраст жителей жилого массива;
+ total_rooms — общее количество комнат в домах жилого массива;
+ total_bedrooms — общее количество спален в домах жилого массива;
+ population — количество человек, которые проживают в жилом массиве;
+ households — количество домовладений в жилом массиве;
+ median_income — медианный доход жителей жилого массива;
+ median_house_value — медианная стоимость дома в жилом массиве;
+ ocean_proximity — близость к океану.

Посмотрим описательную статистику числовых данных:

In [6]:
df.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


**Вывод:** На первый взгляд все значения адекватны, выбросов нет. В дата-сете 20640 строк.

### Обработка пропусков

Посмотрим количество пропусков в дата-сете:

In [7]:
df_pd.isna().mean()

longitude             0.000000
latitude              0.000000
housing_median_age    0.000000
total_rooms           0.000000
total_bedrooms        0.010029
population            0.000000
households            0.000000
median_income         0.000000
median_house_value    0.000000
ocean_proximity       0.000000
dtype: float64

**Вывод:** видим, что пропуски имеются только в одном столбце *total_bedrooms*. Можем заполнить их медианой полученной выше методом describe().

Заполним пропуски в колонке *total_bedrooms* значением 421, полученным из метода describe():

In [74]:
#df = df.na.fill(421)
#print('Количество пропусков в колонке total_bedrooms после заполнения медианой равной 421:', df_pd['total_bedrooms'].isna().mean())

<div class="alert alert-danger">
<b>Комментарий ревьюера ❌:</b> Нам необходимо работать инструментами PySpark. Так же стоит использовать программное заполнение пропусков,вдруг у нас изменятся данные:(

Смотри, как можно сделать:</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Я, кстати, не нашел метода Spark для проверки пропусков. Подскажи пожалуйста) </div>

<div class="alert alert-success">
<b>Комментарий ревьюера V2✔️:</b> Да конечно:) 


+  df.filter(df.total_bedrooms.isNull()).show()

Так можно проверить пропуски в данных, но как ты это делал через pandas тоже нормально. Я имел ввиду, что заполнять пропуски лучше уже методами PySpark, потому что это эффективнее. Можно графики построить через toPandas, но именно изменение значений или работы с данными лучше проводить через pyspark:)</div>

In [75]:
#КОД РЕВЬЮЕРА
# mean = df_housing.select(F.mean('total_bedrooms')).collect()[0][0]
# df_housing = df_housing.na.fill({'total_bedrooms': mean})

In [78]:
df.select('total_bedrooms').rdd.isEmpty()

False

Получим среднее значение колонки total_bedrooms:

In [79]:
mean_total_bedrooms = df.select(F.mean('total_bedrooms')).collect()[0][0]
print('Среднее значение колонки total_bedrooms:', mean_total_bedrooms)

Среднее значение колонки total_bedrooms: 537.8705525375618


Заменим полученным средним значением пропуски в колонке total_bedrooms:

In [80]:
df = df.na.fill({'total_bedrooms': mean_total_bedrooms})

Проверим количество пропусков после заполнения:

In [82]:
df.toPandas().isna().mean()

longitude             0.0
latitude              0.0
housing_median_age    0.0
total_rooms           0.0
total_bedrooms        0.0
population            0.0
households            0.0
median_income         0.0
median_house_value    0.0
ocean_proximity       0.0
dtype: float64

<div class="alert alert-warning">
<b>Комментарий ревьюера ⚠️:</b> Хм...а откуда пропуски? </div>

### Преобразование категориальных значений.

#### Метод StringIndexer

Мы имеем только один категориальный признак - это колонка *ocean_proximity*. Продолжим работать с ней. Преобразуем текстовые значения этой колонки в числовые.

In [11]:
indexer = StringIndexer(inputCol = 'ocean_proximity', 
                        outputCol= 'ocean_proximity_idx') 
df_idx = indexer.fit(df).transform(df)
df_idx = df_idx.drop('ocean_proximity')

In [12]:
df_idx.toPandas().head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_idx
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,3.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,3.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,3.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,3.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,3.0


**Вывод:** мы преобразовали категориальный признак *ocean_proximity* из строковых значений в числовые.

#### Метод OneHotEncoding.

Применим OneHotEncoder к уже трансформированным категориальным признакам:

In [13]:
encoder = OneHotEncoder(inputCol = 'ocean_proximity_idx', 
                        outputCol= 'ocean_proximity_ohe')
df_ohe = encoder.transform(df_idx)

In [14]:
df_ohe.toPandas().head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_idx,ocean_proximity_ohe
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,3.0,"(0.0, 0.0, 0.0, 1.0)"
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,3.0,"(0.0, 0.0, 0.0, 1.0)"
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,3.0,"(0.0, 0.0, 0.0, 1.0)"
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,3.0,"(0.0, 0.0, 0.0, 1.0)"
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,3.0,"(0.0, 0.0, 0.0, 1.0)"


**Вывод:** мы получили трансформированный признак с помощью метода OneHotEncoding.

#### Метод VectorAssembler.

In [15]:
categorical_cols = ['ocean_proximity_ohe']
categorical_assembler = VectorAssembler(inputCols= categorical_cols, outputCol="categorical_features")
df_vect_ceteg = categorical_assembler.transform(df_ohe)

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Все верно! Молодец!</div>

### Преобразование числовых признаков

#### Метод StandartScaler

In [16]:
numerical_cols = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']
numerical_assembler = VectorAssembler(inputCols=numerical_cols, outputCol="numerical_features")
df_SS = numerical_assembler.transform(df_vect_ceteg) 

In [17]:
standardScaler = StandardScaler(inputCol='numerical_features', outputCol="numerical_features_scaled")
df_SS = standardScaler.fit(df_SS).transform(df_SS) 

In [18]:
df_SS.toPandas().head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_idx,ocean_proximity_ohe,categorical_features,numerical_features,numerical_features_scaled
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,3.0,"(0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 1.0)","[-122.23, 37.88, 41.0, 880.0, 129.0, 322.0, 12...","[-61.00726959606955, 17.734477624640412, 3.257..."
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,3.0,"(0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 1.0)","[-122.22, 37.86, 21.0, 7099.0, 1106.0, 2401.0,...","[-61.002278409814444, 17.725114120086744, 1.66..."
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,3.0,"(0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 1.0)","[-122.24, 37.85, 52.0, 1467.0, 190.0, 496.0, 1...","[-61.012260782324645, 17.720432367809913, 4.13..."
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,3.0,"(0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 1.0)","[-122.25, 37.85, 52.0, 1274.0, 235.0, 558.0, 2...","[-61.01725196857974, 17.720432367809913, 4.131..."
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,3.0,"(0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 1.0)","[-122.25, 37.85, 52.0, 1627.0, 280.0, 565.0, 2...","[-61.01725196857974, 17.720432367809913, 4.131..."


<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> С числовыми признаками тоже порядок;)</div>

### Итоговая сборка признаков

Так как обучение модели будем проводить на двух группах признаков: 1.все признаки, 2.только числовые, нам  нужно получить две соответствующие итоговые сборки признаков.

#### Всех признаков

Соберем трансформированные категориальные и числовые признаки признаки с помощью VectorAssembler:

In [19]:
all_features = ['categorical_features','numerical_features_scaled']

final_assembler_all = VectorAssembler(inputCols=all_features, 
                                  outputCol="all_features") 
df_transf_all = final_assembler_all.transform(df_SS)

df_SS.select(all_features).show(3) 

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[3],[1.0])|     [-61.007269596069...|
|       (4,[3],[1.0])|     [-61.002278409814...|
|       (4,[3],[1.0])|     [-61.012260782324...|
+--------------------+-------------------------+
only showing top 3 rows



**Вывод:** мы получили датафрейм из всех признаков готовый для обучения модели.

#### Только числовых признаков

In [20]:
numeric_features = ['numerical_features_scaled']



final_assembler_numeric = VectorAssembler(inputCols=numeric_features, 
                                  outputCol="numeric_features") 
df_transf_numeric = final_assembler_numeric.transform(df_SS)

df_SS.select(numeric_features).show(3) 

+-------------------------+
|numerical_features_scaled|
+-------------------------+
|     [-61.007269596069...|
|     [-61.002278409814...|
|     [-61.012260782324...|
+-------------------------+
only showing top 3 rows



**Вывод:** мы получили датафрейм только из числовых признаков готовый для обучения модели.

## Обучение моделей

### Разделение дата-сета на выборки

#### Все признаки

In [21]:
train, test = df_transf_all.randomSplit([.8,.2], seed=RANDOM_SEED)
print('Количество объектов в обучающей выборке:', train.count())
print('Количество объектов в тестовой выборке:', test.count())

Количество объектов в обучающей выборке: 16554
Количество объектов в тестовой выборке: 4086


#### Только числовые признаки

In [22]:
#df_transf_numeric = df_transf.drop('categorical_features')
train_num, test_num = df_transf_numeric.randomSplit([.8,.2], seed=RANDOM_SEED)
print('Количество объектов в обучающей выборке(числовые признаки):', train_num.count())
print('Количество объектов в тестовой выборке(числовые признаки):', test_num.count())

Количество объектов в обучающей выборке(числовые признаки): 16554
Количество объектов в тестовой выборке(числовые признаки): 4086


### Обучение модели

#### Обучение на всех признаках

По заданию нам нужно предсказать медианную стоимость дома, поэтому целевой переменой будет *median_house_value*:

In [23]:
target = 'median_house_value'

Создаем модель Линейной регрессии со стандартными гиперпараметрами:

In [24]:
lr = LinearRegression(featuresCol = "all_features", labelCol = target)

Создаем сетку для подбора гиперпараметров методо GridSearch:

In [25]:
grid_search = ParamGridBuilder() \
.addGrid(lr.regParam, [0.0, 0.01, 0.1]) \
.addGrid(lr.fitIntercept, [False, True])\
.addGrid(lr.elasticNetParam, [0.5, 1.0]) \
.build()

Создаем оценщик качества модели:

In [26]:
evaluator = RegressionEvaluator(predictionCol='prediction',
                                labelCol='median_house_value')

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Можно было бы просто target передать:)</div>

Создаем объект Кросс-валидации и обучаем его:

In [27]:
cv = CrossValidator(estimator = lr,
                    estimatorParamMaps = grid_search,
                    numFolds = 3,
                    evaluator = evaluator)
cv_model = cv.fit(train)

Получим гиперпараметры наулучшей модели:

In [28]:
cv_model.bestModel.extractParamMap()

{Param(parent='LinearRegression_12841763880c', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'): 2,
 Param(parent='LinearRegression_12841763880c', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'): 0.5,
 Param(parent='LinearRegression_12841763880c', name='epsilon', doc='The shape parameter to control the amount of robustness. Must be > 1.0.'): 1.35,
 Param(parent='LinearRegression_12841763880c', name='featuresCol', doc='features column name'): 'all_features',
 Param(parent='LinearRegression_12841763880c', name='fitIntercept', doc='whether to fit an intercept term'): True,
 Param(parent='LinearRegression_12841763880c', name='labelCol', doc='label column name'): 'median_house_value',
 Param(parent='LinearRegression_12841763880c', name='loss', doc='The loss function to be optimized. Supported options: squaredError, huber. (Default squaredError)'): 'squared

Создадим модель с наилучшими подобранными гиперпараметрами:

In [29]:
lr_best_param = LinearRegression(featuresCol = "all_features",
                                 labelCol = target,
                                 elasticNetParam=0.5,
                                 aggregationDepth=2,
                                 epsilon = 1.35,
                                 fitIntercept = True,
                                 maxIter = 100,
                                 regParam = 0.0,
                                 solver = 'auto')

Обучаем модель со стандартными гиперпараметрами:

In [30]:
model_all_features = lr.fit(train)

Обучаем модель с наилучшими подобранными гиперпараметрами:

In [31]:
model_best_param = lr_best_param.fit(train)

Предсказываем результат:

In [32]:
predictions_all_features = model_all_features.transform(test)

predicted_all_features = predictions_all_features.select("median_house_value", "prediction")
predicted_all_features.show() 

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|           76100.0|171646.12974944245|
|           78300.0|127375.28285254305|
|           58100.0|142073.35934247775|
|           70000.0| 149523.9222311112|
|          128900.0| 207300.0237673507|
|           81300.0|152068.00200302666|
|          109400.0|169228.80627002753|
|           90600.0|181385.70511805313|
|           76900.0|164196.27340133954|
|           75500.0|137998.65138673224|
|           69500.0|111793.80033818958|
|           79600.0|161239.54269032646|
|           90000.0|209951.97280307626|
|           74100.0|156986.13726222143|
|           57500.0| 140226.6404422829|
|          130600.0| 217530.1782125649|
|           72600.0|161738.97243869165|
|           74100.0|165478.50015912717|
|           96100.0|156466.30361996684|
|           90200.0| 153841.0831974405|
+------------------+------------------+
only showing top 20 rows



In [33]:
pred_all_feat_cross = cv_model.transform(test)

In [34]:
pred_model_best_param = model_best_param.transform(test)

**Вывод:** мы получили предсказние *median_house_value* медианной стоимости дома на всех признаках и на трех видах моделей.

#### Обучение только на числовых признаках

Создаем и обучаем модель только на числовых признаках:

In [35]:
lr_numeric = LinearRegression(featuresCol = "numerical_features_scaled", labelCol = target)
model_numeric_features = lr_numeric.fit(train_num)

Предсказываем:

In [36]:
predictions_numeric_features = model_numeric_features.transform(test_num)

predicted_numeric_features = predictions_numeric_features.select("median_house_value", "prediction")
predicted_numeric_features.show() 

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|           76100.0|149425.01001097355|
|           78300.0|  78918.6962352586|
|           58100.0|110388.97210616898|
|           70000.0| 117184.5786861335|
|          128900.0|  174679.027362579|
|           81300.0|  118798.756991236|
|          109400.0|117886.39644013066|
|           90600.0|149207.26721515693|
|           76900.0|132371.30673502292|
|           75500.0|100172.39995250152|
|           69500.0| 60567.62256565364|
|           79600.0| 131541.4082576665|
|           90000.0| 177692.2798895021|
|           74100.0|124469.98564581526|
|           57500.0|105237.25563357025|
|          130600.0|186877.76854795124|
|           72600.0|128340.66694435757|
|           74100.0|139126.50617154222|
|           96100.0|121525.60722429026|
|           90200.0|116327.27074512793|
+------------------+------------------+
only showing top 20 rows



**Вывод:** мы получили предсказние median_house_value медианной стоимости дома только на числовых признаках на модели со стандартными параметрами.

## Оценка качества моделей

Оценим качество моделей по метрикам RMSE, MAE и R2, и сравним точность результата предсказания на всех признаках и только числовых признаках.

### На всех признаках

Оценки качества модели со стандартными гиперпараметрами:

In [37]:
metrics_all_features = model_all_features.summary
RMSE_all = metrics_all_features.rootMeanSquaredError
MAE_all = metrics_all_features.meanAbsoluteError
R2_all = metrics_all_features.r2
print('RMSE модели обученной на всех признаках:', RMSE_all)
print('MAE модели обученной на всех признаках:', MAE_all)
print('Качество модели:', R2_all)

RMSE модели обученной на всех признаках: 69037.84375765473
MAE модели обученной на всех признаках: 50168.6556177571
Качество модели: 0.6427516420087133


Качество модели с применением кросс-валидации:

In [38]:
RMSE_cross = cv_model.bestModel.summary.rootMeanSquaredError
MAE_cross = cv_model.bestModel.summary.meanAbsoluteError
R2_cross = cv_model.bestModel.summary.r2
print(RMSE_cross)
print(MAE_cross)
print(R2_cross)

69037.84375765473
50168.6556177571
0.6427516420087133


Качество модели с наилучшими подобранными гиперпараметрами:

In [39]:
metrics_best_model = model_best_param.summary
RMSE_best = metrics_best_model.rootMeanSquaredError
MAE_best = metrics_best_model.meanAbsoluteError
R2_best = metrics_best_model.r2
print('RMSE модели обученной на всех признаках:', RMSE_best)
print('MAE модели обученной на всех признаках:', MAE_best)
print('Качество модели:', R2_best)                             

RMSE модели обученной на всех признаках: 69037.84375765473
MAE модели обученной на всех признаках: 50168.6556177571
Качество модели: 0.6427516420087133


### Только на числовых признаках

In [40]:
metrics_numeric_features = model_numeric_features.summary
RMSE_num = metrics_numeric_features.rootMeanSquaredError
MAE_num = metrics_numeric_features.meanAbsoluteError
R2_num = metrics_numeric_features.r2
print('RMSE модели обученной только на числовых признаках:', RMSE_num)
print('MAE модели обученной только на числовых признаках:', MAE_num)
print('Качество модели:', R2_num)

RMSE модели обученной только на числовых признаках: 69988.98575138253
MAE модели обученной только на числовых признаках: 51245.2958386423
Качество модели: 0.6328401330161523


## Анализ результатов

В целом модели показали удовлетворительный результат. Мы видим, что качество предсказаний модели обученной на всех признаках выше чем модель обученная только на числовых признаках, но незначительно - около 1%(при абсолютном качестве 64%). Выведем числовую описательную статистику: 

In [41]:
df.describe('median_house_value').collect()

[Row(summary='count', median_house_value='20640'),
 Row(summary='mean', median_house_value='206855.81690891474'),
 Row(summary='stddev', median_house_value='115395.61587441359'),
 Row(summary='min', median_house_value='14999.0'),
 Row(summary='max', median_house_value='500001.0')]

При сравнении RMSE = 69037 со средним 206855, минимальным  14999 и максимальным 500001 значениями, можно сделать вывод, что разброс достаточно большой.

<div class="alert alert-block alert-info">
Экспериментировал с разными моделями: стандартная, с кросс-валидацией, с улучшенными гиперапараметрами, но результаты идентичны. Не могу разобраться в чем причина?
</div>

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> с кросс-валидацией, с улучшенными гиперапараметрами - эта одна и та же модель, ты просто на кросс-валидации уже нашел самую лучшую, а после сам же определил ее же и назвал lr_best_param:)</div>

In [42]:
model_all_features.coefficients.toArray()

array([-138792.06802268, -179071.45119988, -134701.37384463,
       -143019.03953433,  -53579.01347211,  -54608.12122266,
         12840.11943304,   -6891.76449305,   25805.30920697,
        -50709.80255407,   35727.85543855,   72224.70325287])

In [43]:
model_best_param.coefficients.toArray()

array([-138792.06802268, -179071.45119988, -134701.37384463,
       -143019.03953433,  -53579.01347211,  -54608.12122266,
         12840.11943304,   -6891.76449305,   25805.30920697,
        -50709.80255407,   35727.85543855,   72224.70325287])

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Хм...у этих моделей, одинаковые веса получились -> одинаково обучились</div>

<font color='blue'><b>Итоговый комментарий ревьюера</b></font>
<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b>Александр, спасибо за отличный проект! Видно, что долго работал над ним! 
Если есть какие либо вопросы я с удовольствием на них отвечу:) <br> Исправь, пожалуйста, замечания и жду проект на следующую проверку, осталось чуть-чуть:) </div>


<div class="alert alert-info"> <b>Комментарий студента:</b> Александр, спасибо за подробное ревью. Недочеты исправил. Вопрос задал в разделе: заполнения пропусков средним) </div>

<div class="alert alert-success">
<b>Комментарий ревьюера V2✔️:</b>Ответил, надеюсь все понятно:) Удачи в следующих проектах!!!</div>